In [1]:
import numpy as np
import pandas as pd
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt




In [2]:
songs_df=pd.read_csv('tracks.csv')

In [3]:
artists_df=pd.read_csv('artists.csv')

In [4]:
artists_df['genres_upd'] = artists_df['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
songs_df['artists_upd'] = songs_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))


In [5]:
#double checking
songs_df[songs_df['artists_upd'].apply(lambda x: not x)]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd
164,1xEEYhWxT4WhDQdxfPCT8D,Snake Rag,20,194533,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.708,0.3610,...,-11.764,0,0.0441,0.9940,0.883,0.1030,0.902,105.695,4,[]
170,3rauXVLOOM5BlxWqUcDpkg,Chimes Blues,14,170827,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.546,0.1890,...,-15.984,1,0.0581,0.9960,0.908,0.3390,0.554,80.318,4,[]
172,1UdqHVRFYMZKU2Q7xkLtYc,Pickin' On Your Baby,11,197493,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.520,0.1530,...,-14.042,1,0.0440,0.9950,0.131,0.3530,0.319,102.937,4,[]
174,0Vl2DO5U6FjgBpzCtBN3OA,Everybody Loves My Baby,10,152507,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.514,0.1930,...,-13.920,0,0.2380,0.9960,0.199,0.2480,0.665,180.674,4,[]
180,5SvyP1ZeJX1jA7AOZD08NA,Tears,10,187227,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.359,0.3570,...,-11.810,1,0.0511,0.9940,0.819,0.2900,0.753,205.053,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585721,3LrdNQeSEWbHcO06iLv2h1,I'll Sing a Hymn to Mary,11,123995,0,"[""Patrick O'Hagen""]",['1lyT8sdJKWAO5WNX3Cl6SC'],1969-05-01,0.198,0.0766,...,-19.754,1,0.0431,0.9850,0.505,0.1040,0.085,158.869,3,[]
585904,6KzpZArHa1ZOpZRxQhYNoY,Bunga Mawar,5,172440,0,"[""The Mercy's""]",['2QrTiM5ZMIwqn5DQrq9tXc'],1980-01-01,0.545,0.4060,...,-12.144,1,0.0295,0.1520,0.740,0.2220,0.482,123.635,4,[]
586597,6rsm9NTgl9kKPatf7S1yCS,Talk,61,190347,0,"[""Why Don't We""]",['2jnIB6XdLvnJUeNTy5A0J2'],2018-08-31,0.795,0.7000,...,-5.221,0,0.1070,0.0588,0.000,0.0977,0.798,120.002,4,[]
586599,54j7EaJPDmSZYcNYvLSJ78,Trust Fund Baby,66,184460,0,"[""Why Don't We""]",['2jnIB6XdLvnJUeNTy5A0J2'],2018-02-01,0.624,0.7850,...,-3.337,1,0.0498,0.0629,0.000,0.1350,0.593,163.929,4,[]


In [6]:
#removing names of the artists with an apostrophe in their names and list starting with a double quote
songs_df['artists_temp']=songs_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
songs_df['artists_upd']=np.where(songs_df['artists_upd'].apply(lambda x: not x),songs_df['artists_temp'] , songs_df['artists_upd'])

In [7]:
exploded_df=songs_df.explode('artists_upd')

In [8]:
combined_df=exploded_df.merge(artists_df, how='left', left_on='artists_upd',right_on='name', suffixes=('_song','_artists_new'))

In [9]:
combined_df=combined_df.dropna()
combined_df=combined_df[combined_df.genres_upd.map(lambda x: len(x))>0]
combined_df=combined_df.drop('artists_temp',axis=1)
combined_df=combined_df.drop('id_artists',axis=1)


In [10]:
combined_df=combined_df.drop('genres', axis=1)

In [11]:
import itertools 
combined_df_grouped=combined_df.groupby('id_song')['genres_upd'].apply(list).reset_index()
combined_df_grouped['new_genre_list']=combined_df_grouped['genres_upd'].apply(lambda z: list(itertools.chain.from_iterable(z)))
combined_df_grouped

,id_song,genres_upd,new_genre_list
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock]
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[country_quebecois, rock_quebecois]"
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik]
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[candy_pop, new_wave, new_wave_pop, permanent_..."
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[chamame, folclore_salteno, folklore_argentino]"
...,...,...,...
534213,7zzoxJbgjme3366mOp5UnH,"[[dance_pop, pop, pop_rap]]","[dance_pop, pop, pop_rap]"
534214,7zzqq4eJe4zKpYCHckNi1K,"[[bossa_nova, mpb, nova_mpb]]","[bossa_nova, mpb, nova_mpb]"
534215,7zztLmdxNYrjTQ4XqUK8cp,"[[estonian_pop, estonian_rock]]","[estonian_pop, estonian_rock]"
534216,7zztk8Ecg9qDRLX3XNfSyq,"[[classic_swedish_pop, europop, swedish_pop]]","[classic_swedish_pop, europop, swedish_pop]"


In [12]:
combined_df_grouped.rename(columns={"id_song":"id"},inplace=True)

In [13]:
final_df=exploded_df.merge(combined_df_grouped[['id','new_genre_list']],on='id',how='left').dropna()

In [14]:
final_df=final_df.drop('artists',axis=1)
final_df=final_df.drop('artists_temp', axis=1)
final_df=final_df.drop('id_artists',axis=1)


In [15]:
#checking and describing the popularity
final_df['popularity'].describe()

count    691976.000000
mean         27.952637
std          18.944687
min           0.000000
25%          12.000000
50%          27.000000
75%          42.000000
max         100.000000
Name: popularity, dtype: float64

In [16]:
#because max popularity is 100, i'll try and reduce the popularity to a rounded off score out of 10
final_df['popularity_outof10']=final_df['popularity'].apply(lambda x: round(x/10))

In [17]:
final_df['new_genre_list']=final_df['new_genre_list'].apply(lambda x: x if isinstance(x,list) else [])

In [18]:
#only year of the production to be taken into considerations
final_df=final_df.reset_index()
final_df['year']=final_df['release_date'].apply(lambda x: x.split('-')[0])

In [19]:
final_df=final_df.drop(columns='release_date',axis=1)

In [20]:
#checking the list of top artists 
top_artists_df=pd.read_csv('artists_new.csv', low_memory=False)
top_artists_df=top_artists_df.dropna()
top_artists_df

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False
...,...,...,...,...,...,...,...,...,...,...
985013,f6d4e750-9814-463c-ab53-08ae054cc006,Kömma,kömma,Mexico,Mexico,komma rock alternative progressive,alternative rock; Progressive rock; epic; Prog...,0.0,0.0,False
985048,fbf287b4-71d4-463d-a65b-ec724b807051,Marc-Antoine,marc-antoine,Canada,Canada,canada; quebec,canada; Canadian; rnb; quebec; quebecois; r an...,0.0,0.0,False
985348,54a8b63b-ec0b-4e3b-aa29-a1df41933d1f,İlhan Şeşen,İlhan Şeşen,Turkey,Turkey,turkish,pop; turkish; Turkish Pop; spotify,0.0,0.0,False
985886,5276498e-1f61-458e-8d25-13d7378ed0b5,アップアップガールズ（仮）,アップアップガールズ（仮）,Japan,Japan,n; つんく♂プロデュース; ufw,japanese; JPop; idol,0.0,0.0,False


In [21]:
top_artists_df=top_artists_df.drop_duplicates('artist_mb').head(250)

In [22]:
artists_list=top_artists_df['artist_mb']
artists_list=artists_list.tolist()
artists_list = list(dict.fromkeys(artists_list))

In [23]:
final_df=final_df[final_df['artists_upd'].isin(artists_list)]
final_df=final_df.reset_index()
final_df.to_csv('final_df.csv')

In [24]:
#training the model
def train_the_model(df):
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['new_genre_list'].apply(lambda x: " ".join(x)))
    return tfidf_matrix,tfidf

In [25]:
tfidf_matrix,tfidf=train_the_model(final_df)

In [26]:
import pickle

filename_model = 'finalized_model.sav'
pickle.dump(tfidf_matrix, open(filename_model, 'wb'))
filename_object= 'tfidf_object.pk'
pickle.dump(tfidf,open(filename_object,'wb'))